# Linear Regression via Gradient Descent

In [6]:
import numpy as np
from bokeh.plotting import figure, output_notebook,show
output_notebook()

Loading BokehJS ...

### Set up the data

We use the multivariate simulated data from the regression lab, with the target variable being column 1 and the features being columns 2 and 3. We append a column of ones to the data matrix. 

In [7]:
data = np.genfromtxt("data/multivar_simulated/data.csv", skip_header=1, delimiter=",")
Y = data[:, 1].reshape(-1,1)
X = data[:, 2:]
X = np.concatenate([X, np.ones(shape=(X.shape[0], 1))], axis=1)

Find our initial guess and the matrices needed for the gradient.  Choose a learning rate and a tolerance.


In [8]:
M0 = np.random.normal(0,1,size=(X.shape[1],1))

lr = .0001
epsilon=.00001
A = ((X.T) @ Y).reshape(3,1)
D = X.T @ X

Do the gradient descent iteration


In [9]:
MSE0=0
losses = []
for i in range(10000):
    M = M0 - lr*(-2*A+2*(D@M0))
    MSE = np.sum(np.square(Y-(X@M)))
    if np.abs(MSE-MSE0)<epsilon:
        print("converged after {} iterations".format(i))
        break
    M0=M.copy()
    MSE0=MSE
    losses.append(MSE)

converged after 2010 iterations


Compare the results of the gradient descent with the direct solution.

In [10]:
direct_solution = np.linalg.inv(D)@A
print(f"Gradient Descent yields {M.ravel()} while direct computation yields {direct_solution.ravel()}")

Gradient Descent yields [ 1.78971601 -3.47779232  6.05091532] while direct computation yields [ 1.78777492 -3.47899986  6.0608333 ]


Behavior of the MSE over the iterations.


In [11]:

f=figure(title='MSE for gradient descent vs iterations',x_axis_label='Iterations (x10)')
x=list(range(len(losses)))
f.scatter(x=[y/10 for y in x[500::100]],y=losses[500::100])
show(f)

In [12]:

f2=figure(title='MSE vs iterations for gradient descent',x_axis_label='Iterations (x10)')
x=list(range(len(losses)))
f2.scatter(x=[y/10 for y in x[1500::10]],y=losses[1500::10])
show(f2)

## Stochastic Gradient Descent

In stochastic gradient descent, each loop uses just one pair (x,y) from the dataset. So x is just a row of the X matrix, y is an entry from the
y vector, and the error is $\|(y-xM)\|^2$.  The gradient of this is $2x\cdot(y-xM)=2x\cdot y - 2x\cdot xM$.

Stochastic gradient descent avoids having to do any big matrix multiplications, and in particular avoids computing $X^{T}X$ which, if $X$ has many rows, can be prohibitively expensive.

In [13]:
M0 = np.random.normal(0,1,size=(X.shape[1],1))
lr = .0001
epsilon=.00001

In [14]:
losses=[]
for j in range(10000):
    for i in range(X.shape[0]):
        data=X[i,:].reshape(1,X.shape[1])
        target = Y[i,0]
        grad = -2*data*(target-((data@M0)[0,0]))
        M = M0 - lr*grad.T
        M0 = M.copy()
    MSE = np.sum(np.square(Y-(X@M)))
    losses.append(MSE)

In [15]:
print(f"Gradient Descent yields {M.ravel()} while direct computation yields {direct_solution.ravel()}")

Gradient Descent yields [ 1.78866833 -3.48161702  6.05914503] while direct computation yields [ 1.78777492 -3.47899986  6.0608333 ]


Stochastic gradient descent doesn't really converge to a minimum, it tends to move around the minimum point.
To see this let's look at a simple one-variable problem

In [16]:
x=np.linspace(0,10,10)
y=3*x+2+np.random.normal(0,3,size=10)

In [17]:
f5=figure()
f5.scatter(x=x,y=y)
f5.line(x=x,y=3*x+2)
show(f5)

In [18]:
w=np.random.normal(0,1)
b=np.random.normal(0,1)
losses=[]
for j in range(100):
    for i in range(len(x)):
        data=x[i]
        target = y[i]
        wgrad = -2*data*(target-(w*data+b))
        bgrad = -2*(target-(w*data+b))
        w = w - lr*wgrad
        b = b - lr*bgrad
        loss = (target-(w*data+b))**2
        losses.append(loss)


In [19]:
print(f"found slope={w} and intercept={b}")

found slope=3.2564270513780307 and intercept=0.09952766210130136


In [20]:
f6=figure(title="MSE vs iterations in stochastic gradient descent")
f6.scatter(x=list(range(len(losses))),y=losses)
show(f6)

As seen in the graph above, in each inner loop through the data has the MSE bouncing for each particular point bouncing around while the overall trend is downward.  That's because each adjustment produced by a new data point doesn't necessarily move towards the proper minimum. At the later stages the MSE for the different points varies because our estimate isn't equally good for all points. 